In [1]:
from pyplasm import *
from numpy import ones,vstack
from numpy.linalg import lstsq
import numpy as np
from numpy.linalg import norm


Evaluating fenvs.py..
...fenvs.py imported in 0.004764 seconds


In [315]:
"""funzioni matematiche"""

"""trova la retta passante per due punti"""
def equation_from_2_points(p1,p2):
    #print p1,p2
    if(p1[1]==p2[1]):
        result=[0,1,-p1[0]]
        return result
    if(p1[0]==p2[0] and p1[1]!=p2[1]):
        result=[1,0,-p1[0]]
        return result
    points = [(p1[0],p1[1]),(p2[0],p2[1])]
    x_coords, y_coords = zip(*points)
    A = vstack([x_coords,ones(len(x_coords))]).T
    m, c = lstsq(A, y_coords)[0]
    result=[-1*m,1,-1*c]
    return result

"""trovare il punto d'intersezione fra 2 equazioni"""
def equation_solver(eq1,eq2):
    xy1=[eq1[0],eq1[1]]
    xy2=[eq2[0],eq2[1]]
    k=[-1*eq1[2],-1*eq2[2]]
    a=np.array([xy1,xy2])
    b=np.array(k)
    x=np.linalg.solve(a,b)
    x=[x[0],x[1]]
    return x

"""usato da bisector per tornare l'eq della retta passante per la bisettrice"""
def build_bisector_eq(point,angle):
    if(angle==90):
        rect_parallel_to_y
    toReturn=[]
    y=1 
    x=math.tan(-angle)        
    k=-x*point[0]-point[1]
    toReturn.append(x)
    toReturn.append(y)
    toReturn.append(k)
    return toReturn

def rect_parallel_to_y(point):
    toReturn=[]
    k=-point[1]
    x=1
    y=0
    toReturn.append(x)
    toReturn.append(y)
    toReturn.append(k)
    return toReturn

''' 
trova la bisettrice passante per tre punti
Inputs:
    p0,p1,p2 - punti nella forma [x,y]
'''
def bisector(p0,p1,p2):
    print "p0{0}, p1{1}, p2 {2}".format(p0,p1,p2)
    v0 = np.array(p0) - np.array(p1)
    v1 = np.array(p2) - np.array(p1)
    angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
    print "angolo {0}".format(np.degrees(angle))
    return build_bisector_eq(p1,angle/2)
        
def angles_3_point(p0,p1,p2):
    print "p0{0}, p1{1}, p2 {2}".format(p0,p1,p2)
    v0 = np.array(p0) - np.array(p1)
    v1 = np.array(p2) - np.array(p1)
    angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
    return angle
        

"""trova le equazioni delle bisettrici della figura"""
def find_bisector_eq(verts,cells,size,lati):
    result=[]
    vertici={}
    bisectores={}
    i=0
    while(i<len(verts)):
        points=[]
        v=i+1
        i=i+1
        for j in range(0,len(cells)):
            if(len(points)==1):
                points.append(verts[i-1])
            if(cells[j][0]==v):
                points.append(verts[cells[j][1]-1])
            if(cells[j][1]==v):
                points.append(verts[cells[j][0]-1])
        #print points
        index=compute_index(v,size)
        #bisectores[index]=bisector(points[0],points[1],points[2])
        bisectores[index]=bisector_222(lati[v],lati[(v+1)],points[1])
        print bisectores
        vertici[index]=points[1]
    
    result=[bisectores,vertici]
    return result

def bisector_222(r1,r2,p):
    print r1,r2
    a=(r1[0]-r2[0])/(1+r1[0]*r2[0])
    a=abs(a)
    a=math.atan(a)
    print a/2
    k=build_bisector_eq(p,a/2)
    return k


def find_bisector_2rette(r1,r2,v1,v2):
    print "stampiamo gli argomenti r1{0} r2{1} v1{2} v2{3}".format(r1,r2,v1,v2)
    result=[]
    try:
        p1=equation_solver(r1,r2)
        if(r2[2]==0):
            app=r2
            r2=r1
            r1=app
        angles=angles_3_point(v1,p1,v2)
        p_axis=[((1*r2[2])/(r1[0])),0]
        print p_axis
        angle_axis=angles_3_point([0,0],p_axis,p1)
        angle_on_axis=(angles)/2+1*(angle_axis)
        if(abs(np.degrees(angle_on_axis))==90):
            angle_on_axis=90
        newR=build_bisector_eq(p1,angle_on_axis)
        print "la nuova r  e {0}".format(newR)
        result.append(newR)
        result.append(p1)
        return result
    except np.linalg.linalg.LinAlgError as err:
        if 'Singular matrix' in err.message:
            print "le rette sono parallele"
            result.append(r1)
            result.append(v1)
            return result
        
        

def checkPointEquality(v1,v2):

    x1=round(v1[0], 2)
    y1=round(v1[1], 2)
    x2=round(v2[0], 2)
    y2=round(v2[1], 2)
    print "x1y1 ({0},{1}) x2y2 ({2},{3})".format(x1,y1,x2,y2)
    cond= x1==x2 and y1==y2
    return cond
    
    

def compute_index(v,size):
    prev=(v-1)%size
    succ=v%size
    if (prev==0):
        prev=size
    if(succ==0):
        succ=size
    return (prev,succ)

In [316]:
"""trova i punti del cammino in comune"""
def find_common_path_point(bisectores,lati,vertici):
    path_point={}
    app=bisectores.copy()
    i=0
    j=0
    point1=bisectores.keys()[0] 
    for i in range(len(bisectores.values())):
        point2=bisectores.keys()[i]
        print "io ho {1} , tupla da scorrere {0}".format(point2,point1)
        if(point1[1]==point2[0]):
            v1=vertici[point1]
            v2=vertici[point2]
            result=find_bisector_2rette(bisectores[point1],bisectores[point2],v1,v2)
            path_retta=result[0]
            newV=result[1]
            point3=occhicone_method(point1,point2)
            path_point[point3]=path_retta
            del app[point1]
            del app[point2]
            
            while (len(app.values())!=0):
                print "ciclo con j {1} e size {0}".format(len(app.values()),j)
                point2=app.keys()[j]
                print "tuple rimaste {0}".format(app)
                print " t fissa {1}, t da provare {0}".format(point2,point3)
                j=j+1
                if(tuple_equality(point2,point3)):
                    appTuple=point3
                    point3=occhicone_method(point3,point2)
                    print "nuova tupla{0}".format(point3)
                    print app
                    if( app.has_key(point3) ):
                        print "lo contiene"
                        print newV
                        v2=vertici[point2]
                        result=find_bisector_2rette(path_retta,bisectores[point2],newV,v2)
                        if(checkPointEquality(result[1],newV)):
                            newV=result[1]
                            path_retta=result[0]
                            path_point[point3]=path_retta
                        
                            if( app.has_key(point3)):
                                del app[point3]
                            if(app.has_key((point3[1],point3[0]))):
                                del app[(point3[1],point3[0])]
                            point3=appTuple
                        
                            print "app {0}".format(app)
                            j=0
                    else:
                        print point2
                        print "sono qui"
                        v2=vertici[point2]
                        result=find_bisector_2rette(path_retta,bisectores[point3],newV,v2)
                        newRect=result[0]
                        newV=result[1]
                        path_point[point3]=path_retta
                        del app[point2]
                        j=0
                        
                        
            break
    return path_point

"""ritorna la coppia di numeri non in comune di due tuple"""
def occhicone_method(t1,t2):
    print "occhicone su t1 {0} t2 {1}".format(t1,t2)
    if(t1[0]==t2[0]):
        return (t1[1],t2[1])
    if(t1[1]==t2[1]):
        return (t1[0],t2[0])
    if(t1[0]==t2[1]):
        return (t1[1],t2[0])
    if(t1[1]==t2[0]):
        return (t1[0],t2[1])
    
def tuple_equality(t1,t2):
    #print "t1 {0} t2 {1}".format(t1,t2)
    cond1=(t1[0]==t2[1] and t1[1]!=t2[0]) or (t1[1]==t2[0] and t1[0]!=t2[1]) 
    cond2=(t1[1]==t2[1] and t1[0]!=t2[0]) or(t1[0]==t2[0] and t1[1]!=t2[1])
    cond= cond1 or cond2
    return cond

In [317]:
def roofFromLines(verts,cells):
    polyline=SKEL_1(MKPOL([verts,cells,1]))
    lati={}
    vertici={}
    i=0
    j=0
    size=len(verts)
   
    while(i<len(cells)):
        k=i+1
        lato=equation_from_2_points(verts[cells[i][0]-1],verts[cells[i][1]-1])
        lati[k]=lato
        i=i+1
    print lati
    result=find_bisector_eq(verts,cells,size,lati)
    bisectores=result[0]
    vertici=result[1]
        
    path_point=find_common_path_point(bisectores,lati,vertici)
    
    print path_point
    print "vertici: {0} ".format(vertici)
    print "bisettrici: {0} ".format(bisectores)
    print "lati: {0} ".format(lati)
    #VIEW(polyline)
    return 0

In [318]:
#cells=[[1,2],[2,3],[3,4],[4,5],[5,6],[6,1]]
#verts=[[0.,0.],[3.,0.],[2.,1.],[4.,1.],[4.,2.],[1.,1.]]
cells=[[1,2],[2,3],[3,4],[4,1]]
verts=[[0,0],[3,0],[3,1],[0,1]]
roofFromLines(verts,cells)

{1: [0, 1, 0], 2: [1, 0, -3], 3: [0, 1, -3], 4: [1, 0, 0]}
[0, 1, 0] [1, 0, -3]
0.392699081699
{(4, 1): [-0.4142135623730951, 1, 0.0]}
[1, 0, -3] [0, 1, -3]
0.392699081699
{(1, 2): [-0.4142135623730951, 1, 1.2426406871192852], (4, 1): [-0.4142135623730951, 1, 0.0]}
[0, 1, -3] [1, 0, 0]
0.392699081699
{(1, 2): [-0.4142135623730951, 1, 1.2426406871192852], (2, 3): [-0.4142135623730951, 1, 0.24264068711928521], (4, 1): [-0.4142135623730951, 1, 0.0]}


KeyError: 5

In [314]:
eq1=[3,1,9]
eq2=[1,2,8]
print equation_solver(eq1,eq2)
print equation_from_2_points([1,5],[3,4])

[-2.0, -3.0]
[0.500000000000001, 1, -5.5000000000000027]


def find_bisector_2rette(r1,r2,v1,v2):
    print "stampiamo gli argomenti r1{0} r2{1} v1{2} v2{3}".format(r1,r2,v1,v2)
    result=[]
    try:
        p1=equation_solver(r1,r2)
        if(r2[2]==0):
            app=r2
            r2=r1
            r1=app
        angles=angles_3_point(v1,p1,v2)
        p_axis=[((-1*r2[2])/(r1[0])),0]
        print p_axis
        angle_axis=angles_3_point([0,0],p_axis,p1)
        print "angolo fra rette {0}".format(np.degrees(angles))
        print "angolo 2 {0}".format(np.degrees(angle_axis))
        angle_on_axis=(angles)/2-1*(angle_axis)
        print "rispetto all'asse angolo {0}".format(np.degrees(angle_on_axis))
        result.append(build_bisector_eq(p1,angle_on_axis))
        result.append(p1)
        return result
    except np.linalg.linalg.LinAlgError as err:
        if 'Singular matrix' in err.message:
            print "le rette sono parallele"
            result.append(r1)
            result.append(v1)
            return result
        